> ## Make a copy of this notebook (File menu -> Make a Copy...)

##### Throughout this week's homework, use your LUdet(A) function that you wrote in lab.

### Homework Question 1

1. Implement a function called *Cramers(A,v)* that solves the matrix equation $Ax=v$ using Cramer's Rule. If you need a reminder of Cramer's Rule, [see here](https://en.wikipedia.org/wiki/Cramer%27s_rule).<br><br>

1. Consider the matrix equation $Ax=v$ where <br><br>
   
   $$A=\begin{bmatrix}-4 & 1 & -9 &  4 &  4 &  1 &  9 & -6\\
-1 & -3 & -8 & -2 & -4 &  4 &  2 &  7\\
 3 &  6 & -8 &  1 &  0 &  1 &  2 &  5\\
-10 & -4 & -1 & -4 &  2 & -8 &  3 &  2\\
-6 &  7 & -4 & -6 &  2 &  1 & -4 &  8\\
 6 &  6 &  5 &  8 & -7 &  9 & -1 &  9\\
 1 & -10 &  2 & -6 & -8 & -9 & -4 &  0\\
 6 &  1 & -4 &  3 &  8 &  3 &  3 &  5\end{bmatrix}\mbox{ and }v=\begin{bmatrix} 28 \\ 40 \\ 45 \\ -18 \\ 26 \\ 114 \\ -125 \\ 102\end{bmatrix}$$<br><br> 
 
      Solve these using both your Cramer's Rule function and previous solution techniques. Run timing comparisons and comment on your results. For your convenience, here is code for those arrays:
      
```python
A = np.array([[4 , 1 , -9 ,  4 ,  4 ,  1 ,  9 , -6],[1 , -3 , -8 , -2 , -4 ,  4 ,  2 ,  7],[3 ,  6 , -8 ,  1 ,  0 ,  1 ,  2 ,  5],[10 , -4 , -1 , -4 ,  2 , -8 ,  3 ,  2],[6 ,  7 , -4 , -6 ,  2 ,  1 , -4 ,  8],[6 ,  6 ,  5 ,  8 , -7 ,  9 , -1 ,  9],[1 , -10 ,  2 , -6 , -8 , -9 , -4 ,  0],[6 ,  1 , -4 ,  3 ,  8 ,  3 ,  3 ,  5]])

v=np.array([28,40,45,-18,26,114,-125,102])
```
      

In [77]:
import numpy as np
from rref import rref

from Qiureferencefunctions import LU,swaprows,rowaddmult, fwdsub, backsub
def Cramers(A,v):
    d = det(A)
    rows = A.shape[0]
    ret = np.zeros((rows))
    for i in range (0,rows):
        C = np.zeros((rows,rows))
        C[:,:i] = A[:,:i]
        C[:,i] = v
        C[:,i+1:] = A[:,i+1:]
        ret[i] = det(C)/d
    return ret
A = np.array([[-4 , 1 , -9 ,  4 ,  4 ,  1 ,  9 , -6],[-1 , -3 , -8 , -2 , -4 ,  4 ,  2 ,  7],[3 ,  6 , -8 ,  1 ,  0 ,  1 ,  2 ,  5],[-10 , -4 , -1 , -4 ,  2 , -8 ,  3 ,  2],[-6 ,  7 , -4 , -6 ,  2 ,  1 , -4 ,  8],[6 ,  6 ,  5 ,  8 , -7 ,  9 , -1 ,  9],[1 , -10 ,  2 , -6 , -8 , -9 , -4 ,  0],[6 ,  1 , -4 ,  3 ,  8 ,  3 ,  3 ,  5]])
v=np.array([28,40,45,-18,26,114,-125,102])
sol = Cramers(A,v)
print(sol)
print(A@sol)


[-0.  1.  2.  3.  4.  5.  6.  7.]
[  28.   40.   45.  -18.   26.  114. -125.  102.]


In [52]:
%timeit Cramers(A,v)
%timeit LUSolve(A,v)
#The Cramers method is a lot slower compared to using LU decomposition to solve for v. 

5.07 s ± 202 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
441 µs ± 52.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [47]:
def LUdet(A):
    rows,cols = A.shape
    copy = A.copy()
    pivotcol = 0
    pivotrow = 0
    i = 1
    numswaps =0
    zero = np.zeros((rows,cols))
    perm = np.eye(rows)
    cool = (copy,zero,perm) #U,L,P, 
    while((pivotcol<cols) & (pivotrow<rows)):
        while(i<rows):
            maxe = np.argmax(abs(copy[pivotrow:,pivotcol])) +pivotrow
            if (maxe > pivotrow):
                swaprows(perm,maxe,pivotrow)
                swaprows(zero,maxe,pivotrow)
                numswaps+=1
                copyrow = (copy[pivotrow]).copy();
                copy[pivotrow] = (copy[maxe]).copy();
                copy[maxe] = copyrow;
            multval = (-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])
            rowaddmult(copy,pivotrow,i,(multval))
            zero[i,pivotrow] = -multval
            i+=1
        pivotcol+=1
        pivotrow+=1
        i = pivotrow+1
    np.fill_diagonal(zero,1.)
    Udet = np.prod(np.diag(copy))
    numswaps = numswaps%2
    Pdet = (-1)**numswaps
    return Udet*Pdet
def LUSolve(A,v):
    U,L,P = LU(A)
    v = P@v
    y = fwdsub(L,v)
    x = backsub(U,y)
    return x

### Homework Question 2

The inverse $A^{-1}$ of a matrix can be found using the *cofactor matrix*. If $A$ is a matrix<br><br> $$A=\begin{bmatrix}a_{1,1} & \cdots & a_{1,n} \\ \vdots & \ddots & \vdots \\ a_{n,1} & \cdots & a_{n,n}\end{bmatrix}$$<br><br> then its *cofactor matrix* is defined to be $$adj(A)=\begin{bmatrix}C_{1,1} & \cdots & C_{1,n} \\ \vdots & \ddots & \vdots \\ C_{n,1} & \cdots & C_{n,n}\end{bmatrix}^T,$$<br><br> where $C_{i,j}$ is the $(i,j)$ cofactor of $a_{i,j}$ (note the transpose sign!).<br><br>

1. Find by hand the cofactor matrix of <br><br>$$A=\begin{bmatrix} -2 & 11 & 48 \\-2 & 11 & 49 \\3 & -16 & -70
\end{bmatrix}.$$<br><br>
1. Find the determinant of $A$, and show that $A\cdot adj(A)=det(A)\cdot I_3$ in this case (where $I_3$ is the $3\times 3$ identity matrix).<br><br>
1. This property is true in general. Use this to write a function called `detInv(A)` that computes the inverse of $A$ using determinants. Try to avoid computing cofactors twice!<br><br>
1. Use your function to compute the inverse of the matrix from the first part of this question above.<br><br>
1. Is this an efficient way to compute inverses? Experimentally show that once again, using row reduction is far better. (See [here](https://www.mathsisfun.com/algebra/matrix-inverse-row-operations-gauss-jordan.html) if you need a reminder as to how to find inverses using row reduction. Feel free to use the `rref(A,tol)` function.)

In [80]:
A = np.array([[-2,11,48],[-2,11,49],[3,-16,-70]])
C = np.array([[14,7,-1],[2,-4,1],[11,2,0]])
C= C.T
d = det(A)
print(A@C)
print(d*np.eye(3))

[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [81]:
def detInv(A): 
    rows,cols = A.shape
    i = 0
    inv = np.zeros ((rows,cols))
    for i in range (0,rows):
        for j in range (0,cols):
            B = np.delete(A,i, axis=0)
            B = np.delete(B,j, axis=1)
            inv[i,j] = (-1)**(i+j)* det(B)
    return inv.T

In [88]:
print(detInv(A))
rows,cols = np.shape(A)
AA = np.zeros((rows,cols+cols))
AA[0,cols]=1
AA[1,cols+1]=1
AA[2,cols+2]=1
AA[0:rows,0:cols] = A
print(rref(AA,10**-10))

[[14.  2. 11.]
 [ 7. -4.  2.]
 [-1.  1.  0.]]
[[ 1.  0.  0. 14.  2. 11.]
 [ 0.  1.  0.  7. -4.  2.]
 [-0. -0.  1. -1.  1. -0.]]


In [91]:
%timeit detInv(A)
%timeit rref(AA,10**-10)

387 µs ± 6.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
156 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


It is far more efficient to do row reduction in order to find the inverse of A, around 2x faster than using cofactors to find the inverse of a matrix.

### Homework Question 3

It is often said that determinants are one of the least useful ways to do numerical computation with matrices. Do some research online and read a paper or two, then write a paragraph summarizing why this is the case. Include references.

Determinants are often disreputed due to their extremely computationally inefficient nature. Methods to compute the determinant or to use the determinant to find another value/matrix (like using Cramer's rule to find the inverse of a matrix) are extremely inefficient, far more so than using simpler methods to compute the desired value. For instance, Axler denigrates the use of determinants to find eigenvalues, which we have yet to study. However, he demonstrates that it is completely possible to calculate these values sans the use of determinants, which typically take a very long time to compute.He states that there is virtually no linear algebra problem that can only be solved with the determinant value; even Cramer's rule, which heavily implements the calculation of determinants,"is completely impractical" (Axler).

Axler, S. (1995). Down with determinants!. The American mathematical monthly, 102(2), 139-154.